In [1]:
cd 'drive/MyDrive/DAN20-Data Analytics Final Project/Data/Model_Data'

/content/drive/MyDrive/DAN20-Data Analytics Final Project/Data/Model_Data


In [2]:
ls

description.csv  movies.csv  ratings.csv


In [3]:
#Libraries that we have used 
import pandas as pd 
import numpy as np
import nltk
from nltk.corpus import 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS 
from nltk.corpus import stopwords
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def sort_movies_by_year(li):
    def merge(a,l,mid,r):
        n1=mid-l+1
        n2=r-(mid+1)+1
        L=[a[i+l] for i in range(n1)]
        R=[a[i+mid+1] for i in range(n2)]
        i,j,k=0,0,l
        while(i<n1 and j<n2):
            if int(L[i][-5:-1])>int(R[j][-5:-1]) :
                a[k]=L[i]
                i+=1
            else:
                a[k]=R[j]
                j+=1
            k+=1
        while(i<n1):
            a[k]=L[i]
            i+=1
            k+=1
        while(j<n2):
            a[k]=R[j]
            j+=1
            k+=1
    def merge_sort(a,l,r):
        if l==r:
            return
        mid=(l+r)//2
        merge_sort(a,l,mid)
        merge_sort(a,mid+1,r)
        merge(a,l,mid,r)

    merge_sort(li,0,len(li)-1)

In [12]:
#Importing files that we want to use
movies = pd.read_csv("movies.csv")
ratings= pd.read_csv("ratings.csv")
description= pd.read_csv("description.csv")

#selecting columns that help in the pridiction of thye movies
description_j = description[["imdb_title_id","original_title","description","language","country"]]
imdb = pd.DataFrame(description_j)

#Selecting rows with english as language
imdb_eng = imdb[imdb["language"] == "English" ]
imdb_eng.tail()
imdb_eng.dropna(inplace= True)

#selecting last 15k movies for computation ease
imdb_2 = imdb_eng.iloc[15000:]

#setting the title as index for ease of reference
test = imdb_2.reset_index(drop = True)
test = test.drop("imdb_title_id",axis = 1)
test.set_index("original_title",drop = True,inplace = True)

#filtering stopwords and implementing stemming and lemmatization
stop_w = set(stopwords.words('english'))
stop_words = []
for word in stop_w:
  stop_words.append(word)
lemmatizer = WordNetLemmatizer() 
ps = PorterStemmer() 
i=0
stop_copy=set(stop_words)

i = 0
for text in test["description"]:
  text=str(text)
  text=set(text.split(" "))
  text=text-stop_copy
  a = list(text)
  w = []
  for word in a:
    word_s = ps.stem(word)
    word_l = lemmatizer.lemmatize(word_s)
    w.append(word_l)
  sentence = ""
  for word in w:
    sentence = sentence + word
    sentence = sentence + " "
  test["description"][i]= sentence
  i=i+1

#dropping rows for null values as a part of data preprocessing
test.dropna(inplace= True)
test.isnull().values.any()

#calculating TfIdf using sklearn lib and finding cosine similarity between one movie to other 
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df = 3 , max_features = None , strip_accents = "unicode", analyzer = "word" , ngram_range = (1,3))
test2 = test
test2["description"] = test2["description"].fillna(" ")
tfv_matrix = tfv.fit_transform(test2["description"])
tfv_matrix.shape
from sklearn.metrics.pairwise import cosine_similarity
sigmoid_sim = cosine_similarity(tfv_matrix, tfv_matrix)

dic1 = {}
dic2 = {}
indices = pd.DataFrame(test.index)
for i,text in enumerate(indices["original_title"]):
  dic1[text] = i
  dic2[i] = text
indices.head()
#sigmoid_sim[0]

#implimenting recommender_system and giving top ten movies
def recommend_system(title,sigmoid_sim):
  index = dic1[title]
  sig_scores = list(enumerate(sigmoid_sim[index]))
  scores = sorted(sig_scores,key= lambda x : x[1],reverse=True)
  scores = scores[1:11]
  print(scores)
  movies_indics = [i[0] for i in scores]
  movies = [dic2[i] for i in movies_indics]
  return movies


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [15]:
a = recommend_system("The Samaritan",sigmoid_sim)
print(a)

[(9846, 0.2327765117046871), (5024, 0.1391305946541672), (3042, 0.13485905522399785), (139, 0.13481761325088282), (10879, 0.13080769281842122), (3515, 0.13039893526893917), (17411, 0.12862161710105136), (5327, 0.12786039669923827), (6923, 0.12658867017698333), (7281, 0.12531276271042419)]
['First Man', 'Bringing Down the House', 'Fortress 2', "Nobody's Fool", 'Henry & Me', 'The Awakening of Gabriella', 'Arlo: The Burping Pig', 'The Brown Bunny', 'South of Heaven', 'Hit Me']


In [10]:
#selecting indian movies and preprocessing data
imdb_indian = imdb[imdb["country"] == "India"]
imdb_indian.dropna(inplace= True)

#imdb_indian.tail(100)
imdb_telugu = imdb_indian[imdb_indian["language"] == "Telugu"]
#imdb_telugu.tail(100)

#setting the title as index for ease of reference
test = imdb_indian.reset_index(drop = True)
test = test.drop("imdb_title_id",axis = 1)
test.set_index("original_title",drop = True,inplace = True)

#filtering stopwords and implementing stemming and lemmatization
stop_w = set(stopwords.words('english'))
stop_words = []
for word in stop_w:
  stop_words.append(word)
lemmatizer = WordNetLemmatizer() 
ps = PorterStemmer() 
i=0
stop_copy=set(stop_words)

i = 0
for text in test["description"]:
  text=str(text)
  text=set(text.split(" "))
  text=text-stop_copy
  a = list(text)
  w = []
  for word in a:
    word_s = ps.stem(word)
    word_l = lemmatizer.lemmatize(word_s)
    w.append(word_l)
  sentence = ""
  for word in w:
    sentence = sentence + word
    sentence = sentence + " "
  test["description"][i]= sentence
  i=i+1

#dropping rows for null values as a part of data preprocessing
test.dropna(inplace= True)
test.isnull().values.any()

#calculating TfIdf using sklearn lib and finding cosine similarity between one movie to other 
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df = 3 , max_features = None , strip_accents = "unicode", analyzer = "word" , ngram_range = (1,3))
test2 = test
test2["description"] = test2["description"].fillna(" ")
tfv_matrix = tfv.fit_transform(test2["description"])
tfv_matrix.shape
from sklearn.metrics.pairwise import cosine_similarity
sigmoid_sim = cosine_similarity(tfv_matrix, tfv_matrix)


dic1 = {}
dic2 = {}
indices = pd.DataFrame(test.index)
for i,text in enumerate(indices["original_title"]):
  dic1[text] = i
  dic2[i] = text
#print(dic1["Andaz"])
#print(dic2[1])
indices.head()
#sigmoid_sim[0]

#implementing recommender system and giving top 10 movies
def recommend_system(title,sigmoid_sim):
  index = dic1[title]
  sig_scores = list(enumerate(sigmoid_sim[index]))
  scores = sorted(sig_scores,key= lambda x : x[1],reverse=True)
  scores = scores[1:11]
  movies_indics = [i[0] for i in scores]
  movies = [dic2[i] for i in movies_indics]
  return movies

a = recommend_system("Seema Raja",sigmoid_sim)
print(a)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


['Gurkha', 'Vellinakshatram', 'Gang Leader', 'Valu', 'Ishq', 'Laali Ki Shaadi Mein Laaddoo Deewana', 'Oosaravelli', 'Kshay', 'Chandamama Kathalu', 'Khel Khel Mein']


In [8]:
#importing NearestNeighbours and initializig lemmatizer
from sklearn.neighbors import NearestNeighbors
lemmatizer = WordNetLemmatizer()
li = []

#merging movies dataset with ratings
dataset = movies.merge(ratings)

#selecting required attributes
dataset = dataset.loc[:,["userId","movieId","title","genres","rating"]]
df_ratings = dataset.loc[:,["title","rating"]].groupby("title").mean()
genres = dataset["genres"]

#preprocessing generes part
for i in range(len(genres)):
    temp = genres[i].split("|")
    for j in range(len(temp)):
        temp[j] = lemmatizer.lemmatize(temp[j])
    li.append(" ".join(temp)) 

#initializing countvectorizer and fitting data
cv = CountVectorizer()
X = cv.fit_transform(li).toarray()

#joining generes to each movie and user part
genres = pd.DataFrame(X,columns=cv.get_feature_names())
dataset = dataset.iloc[:,:-2]
new_dataset = dataset.join(genres)
users = new_dataset.drop(["movieId","title"],axis=1)
users_moviemat = users.groupby("userId").sum()
X = users_moviemat.iloc[:,:].values

#implimenting NearestNeighbors and getting similar movies
classifier = NearestNeighbors()
classifier.fit(X)
uid = int(input("Enter User Id "))
li = classifier.kneighbors([X[uid-1]],n_neighbors=30,return_distance=False)
current_user = new_dataset.loc[new_dataset["userId"]==li[0][0],:]["title"].values
similar_user = new_dataset.loc[new_dataset["userId"]==li[0][1],:]["title"].values
movies_list = [movie for movie in similar_user if movie not in current_user]
sort_movies_by_year(movies_list)
for i in range(len(movies_list)):
    movies_list[i] = (movies_list[i], df_ratings['rating'][df_ratings.index == movies_list[i]].values[0])
print("Recommended Movies are: ")
print(movies_list)

Enter User Id 108
Recommended Movies are: 


[('Harry Potter and the Deathly Hallows: Part 1 (2010)', 3.7580645161290325),
 ('Black Swan (2010)', 3.9655172413793105),
 ('Devil (2010)', 3.0714285714285716),
 ('Easy A (2010)', 3.3461538461538463),
 ('Social Network, The (2010)', 3.875),
 ('Machete (2010)', 3.3636363636363638),
 ('Scott Pilgrim vs. the World (2010)', 4.0),
 ('Expendables, The (2010)', 3.125),
 ('Salt (2010)', 3.3125),
 ('Karate Kid, The (2010)', 3.3333333333333335),
 ('Inception (2010)', 4.189320388349515),
 ('Toy Story 3 (2010)', 4.142857142857143),
 ('Get Him to the Greek (2010)', 3.15),
 ('Iron Man 2 (2010)', 3.6451612903225805),
 ('Death at a Funeral (2010)', 2.75),
 ('Kick-Ass (2010)', 3.857142857142857),
 ('How to Train Your Dragon (2010)', 3.757142857142857),
 ("She's Out of My League (2010)", 3.0),
 ("Valentine's Day (2010)", 3.0),
 ('Avatar (2009)', 3.856060606060606),
 ('Invictus (2009)', 3.5833333333333335),
 ('Blind Side, The  (2009)', 3.9347826086956523),
 ('Ninja Assassin (2009)', 3.2857142857142856),
